In [ ]:

!pip install Flask pandas scikit-learn numpy scikit-surprise


In [ ]:

import pandas as pd

# Sample ratings dataset (userId, movieId, rating)
ratings_data = {
    'userId': [1, 1, 1, 2, 2, 2],
    'movieId': [1, 2, 3, 1, 2, 3],
    'rating': [5, 4, 3, 4, 5, 2]
}

# Create a DataFrame
ratings = pd.DataFrame(ratings_data)

# Show the data
ratings.head()


In [ ]:

from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split

# Prepare the dataset in a format suitable for scikit-surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Split the data into training and test sets
trainset, testset = train_test_split(data, test_size=0.2)

# Train an SVD model
model = SVD()
model.fit(trainset)

# Evaluate the model
predictions = model.test(testset)
predictions[:5]  # Show first 5 predictions


In [ ]:

from flask import Flask, jsonify, request

app = Flask(__name__)

# Load the dataset and model
trainset = data.build_full_trainset()
model.fit(trainset)

@app.route('/')
def home():
    return "Welcome to the Movie Recommender!"

@app.route('/recommend', methods=['GET'])
def recommend():
    user_id = int(request.args.get('user_id'))
    movie_id = int(request.args.get('movie_id'))
    
    # Predict the rating for a given user and movie
    predicted_rating = model.predict(user_id, movie_id).est
    return jsonify({'predicted_rating': predicted_rating})

if __name__ == '__main__':
    app.run(debug=True)
